In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
data = pd.read_csv("./../Data/DontOverfit/train.csv")
test_data = pd.read_csv("./../Data/DontOverfit/test.csv")
__id__ = data.pop("id")

In [6]:
target_y = data.pop("target")

In [7]:
data.shape

(250, 300)

In [42]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.linear_model  import LogisticRegression

In [43]:
X = data.values

In [44]:
y = target_y.values.reshape(-1,1)

In [45]:
RFselector = SelectFromModel(estimator=RandomForestClassifier()).fit(X, y)
GBMselector = SelectFromModel(estimator=GradientBoostingClassifier()).fit(X, y)
LRselector = SelectFromModel(estimator=LogisticRegression()).fit(X, y)

/root/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/from_model.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/root/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning

In [46]:
columns = data.columns

In [47]:
RF_Selected = columns[RFselector.get_support()]
len(RF_Selected)

113

In [48]:
LR_Selected = columns[LRselector.get_support()]
len(LR_Selected)

124

In [49]:
GBM_Selected = columns[GBMselector.get_support()]
len(GBM_Selected)

69

In [50]:
len(list(set(RF_Selected) & set(LR_Selected) & set(GBM_Selected) ) )

18

In [183]:
object1 = pd.Series(np.random.choice(list("ABCDE") ,
                                     10000 , p = [0.4,0.35,0.1,0.1,0.05] ))
object2 = pd.Series(np.random.choice(["aa","bb","cc",np.nan,"ee"] , 
                                     10000 , p = [0.4,0.35,0.1,0.05, 0.10] ))
object3 = pd.Series(np.random.choice(["F","G",np.nan,"I","J"] , 
                                     10000 , p = [0.5,0.25,0.1,0.1,0.05] ))
object4 = pd.Series(np.random.choice(["F",np.nan,"H","I","J"] , 
                                     10000 , p = [0.6,0.15,0.1,0.1,0.05] ))
object5 = pd.Series(np.random.choice([np.nan,"G","H","I","J"] , 
                                     10000 , p = [0.3,0.45,0.1,0.1,0.05] ))
object6 = pd.Series(np.random.choice(["F","G","H","I",np.nan] , 
                                     10000 , p = [0.4,0.05,0.4,0.1,0.05] ))

obj = pd.concat([object1 , object2 , object3,
                 object4 , object5 , object6
                ], axis = 1)
obj = obj.astype("category")

In [188]:
obj["OBJ1"].dtype == "category"

True

In [184]:
obj.columns = ["OBJ1", "OBJ2","OBJ3" , "OBJ4", "OBJ5","OBJ6"]
x = pd.get_dummies(obj , dummy_na= True ).values

In [167]:
x.shape

(10000, 36)

In [168]:
emb = RandomTreesEmbedding(max_leaf_nodes  = 2 , max_depth = 2 )

In [169]:
print(dir(emb))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'base_estimator', 'bootstrap', 'class_weight', 'criterion', 'decision_path', 'estimator_params', 'feature_importances_', 'fit', 'fit_transform', 'get_params', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_

In [170]:
emb.fit(x)

/root/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomTreesEmbedding(max_depth=2, max_leaf_nodes=2, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=10, n_jobs=None, random_state=None,
                     sparse_output=True, verbose=0, warm_start=False)

In [171]:
embedding = emb.transform(x).toarray()

In [172]:
embedding.shape

(10000, 20)

In [182]:
np.arange(0,250).tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [173]:
np.sum(embedding, axis = 0)

array([9037.,  963., 9044.,  956., 9037.,  963., 9011.,  989., 9018.,
        982., 9003.,  997., 9004.,  996., 9032.,  968., 9522.,  478.,
       9037.,  963.])

In [117]:
embedding.shape

(250, 100)

In [178]:
from sklearn.manifold import Isomap

In [179]:
embedding = Isomap(n_components=2)

In [181]:
#embedding.fit_transform(x)